# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 13 2022 8:30AM,246209,10,MSP213890,"Methapharm, Inc.",Released
1,Sep 13 2022 8:30AM,246209,10,MSP213891,"Methapharm, Inc.",Released
2,Sep 13 2022 8:30AM,246209,10,MSP213893,"Methapharm, Inc.",Released
3,Sep 13 2022 8:30AM,246209,10,MSP213894,"Methapharm, Inc.",Released
4,Sep 13 2022 8:30AM,246209,10,MSP213895,"Methapharm, Inc.",Released
5,Sep 13 2022 8:30AM,246209,10,MSP213896,"Methapharm, Inc.",Released
6,Sep 13 2022 8:30AM,246209,10,MSP213897,"Methapharm, Inc.",Released
7,Sep 13 2022 8:30AM,246209,10,MSP213899,"Methapharm, Inc.",Released
8,Sep 13 2022 8:30AM,246209,10,MSP213900,"Methapharm, Inc.",Released
9,Sep 13 2022 8:30AM,246209,10,MSP213902,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,246210,Released,5
26,246211,Released,9
27,246213,Released,11
28,246214,Released,1
29,246224,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246210,NaN,NaN,5.0
246211,NaN,NaN,9.0
246213,NaN,NaN,11.0
246214,NaN,NaN,1.0
246224,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246106,0.0,1.0,0.0
246131,25.0,9.0,1.0
246148,9.0,10.0,2.0
246155,0.0,0.0,1.0
246156,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246106,0,1,0
246131,25,9,1
246148,9,10,2
246155,0,0,1
246156,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246106,0,1,0
1,246131,25,9,1
2,246148,9,10,2
3,246155,0,0,1
4,246156,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246106,,1,
1,246131,25,9,1
2,246148,9,10,2
3,246155,,,1
4,246156,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc."
13,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc."
18,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation
27,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation
38,Sep 13 2022 8:30AM,246224,10,RHL Supplies
39,Sep 12 2022 4:12PM,246214,16,"SHL Pharma, LLC"
40,Sep 12 2022 3:26PM,246198,22,"NBTY Global, Inc."
41,Sep 12 2022 3:26PM,246197,22,"NBTY Global, Inc."
42,Sep 12 2022 3:19PM,246193,18,Innova Softgel LLC
43,Sep 12 2022 3:19PM,246194,102,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc.",,,13
1,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",,,5
2,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,,,9
3,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,,,11
4,Sep 13 2022 8:30AM,246224,10,RHL Supplies,,,1
5,Sep 12 2022 4:12PM,246214,16,"SHL Pharma, LLC",,,1
6,Sep 12 2022 3:26PM,246198,22,"NBTY Global, Inc.",,,1
7,Sep 12 2022 3:26PM,246197,22,"NBTY Global, Inc.",,,1
8,Sep 12 2022 3:19PM,246193,18,Innova Softgel LLC,,,1
9,Sep 12 2022 3:19PM,246194,102,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc.",13,,
1,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",5,,
2,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,9,,
3,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,11,,
4,Sep 13 2022 8:30AM,246224,10,RHL Supplies,1,,
5,Sep 12 2022 4:12PM,246214,16,"SHL Pharma, LLC",1,,
6,Sep 12 2022 3:26PM,246198,22,"NBTY Global, Inc.",1,,
7,Sep 12 2022 3:26PM,246197,22,"NBTY Global, Inc.",1,,
8,Sep 12 2022 3:19PM,246193,18,Innova Softgel LLC,1,,
9,Sep 12 2022 3:19PM,246194,102,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc.",13,,
1,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",5,,
2,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,9,,
3,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,11,,
4,Sep 13 2022 8:30AM,246224,10,RHL Supplies,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc.",13.0,NaN,NaN
1,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",5.0,NaN,NaN
2,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,9.0,NaN,NaN
3,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,11.0,NaN,NaN
4,Sep 13 2022 8:30AM,246224,10,RHL Supplies,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 13 2022 8:30AM,246209,10,"Methapharm, Inc.",13.0,0.0,0.0
1,Sep 13 2022 8:30AM,246210,10,"Methapharm, Inc.",5.0,0.0,0.0
2,Sep 13 2022 8:30AM,246211,10,ISDIN Corporation,9.0,0.0,0.0
3,Sep 13 2022 8:30AM,246213,10,ISDIN Corporation,11.0,0.0,0.0
4,Sep 13 2022 8:30AM,246224,10,RHL Supplies,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1723403,54.0,8.0,3.0
102,246194,1.0,0.0,0.0
16,492370,6.0,0.0,0.0
18,246193,1.0,0.0,0.0
19,984654,6.0,10.0,28.0
20,984595,9.0,16.0,31.0
22,492395,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1723403,54.0,8.0,3.0
1,102,246194,1.0,0.0,0.0
2,16,492370,6.0,0.0,0.0
3,18,246193,1.0,0.0,0.0
4,19,984654,6.0,10.0,28.0
5,20,984595,9.0,16.0,31.0
6,22,492395,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,54.0,8.0,3.0
1,102,1.0,0.0,0.0
2,16,6.0,0.0,0.0
3,18,1.0,0.0,0.0
4,19,6.0,10.0,28.0
5,20,9.0,16.0,31.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,54.0
1,102,Released,1.0
2,16,Released,6.0
3,18,Released,1.0
4,19,Released,6.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,16,18,19,20,22
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,28.0,31.0,0.0
Executing,8.0,0.0,0.0,0.0,10.0,16.0,0.0
Released,54.0,1.0,6.0,1.0,6.0,9.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,16,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,28.0,31.0,0.0
1,Executing,8.0,0.0,0.0,0.0,10.0,16.0,0.0
2,Released,54.0,1.0,6.0,1.0,6.0,9.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,16,18,19,20,22
0,Completed,3.0,0.0,0.0,0.0,28.0,31.0,0.0
1,Executing,8.0,0.0,0.0,0.0,10.0,16.0,0.0
2,Released,54.0,1.0,6.0,1.0,6.0,9.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()